## Hello Ther

In [ ]:
print("Hello World!")

In [ ]:
def hello(there):
    """Github-flavoured theme"""

In [2]:
!jupyter --version

jupyter core     : 4.6.3
jupyter-notebook : 6.0.3
qtconsole        : not installed
ipython          : 7.13.0
ipykernel        : 5.1.4
jupyter client   : 6.0.0
jupyter lab      : 2.0.1
nbconvert        : 5.6.1
ipywidgets       : 7.5.0
nbformat         : 5.0.4
traitlets        : 4.3.3


In [3]:
!ipython --version

7.13.0


In [1]:
!start .

In [4]:
!pip3 show notebook

Name: notebook
Version: 6.0.2
Summary: A web-based notebook environment for interactive computing
Home-page: http://jupyter.org
Author: Jupyter Development Team
Author-email: jupyter@googlegroups.com
License: BSD
Location: c:\users\pachacho\miniconda3\lib\site-packages
Requires: prometheus-client, nbconvert, tornado, jinja2, jupyter-core, Send2Trash, pyzmq, jupyter-client, ipython-genutils, terminado, traitlets, ipykernel, nbformat
Required-by: widgetsnbextension, jupyterlab, jupyterlab-server, jupyter
